In [24]:
import sqlite3
import random
import hashlib
import requests
from datetime import datetime
import requests
import os
from docx import Document
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

import warnings
warnings.filterwarnings('ignore')

In [25]:
# --------------- USAJobs API Configuration ---------------
USAJOBS_API_KEY = "HiVRyd6XtR3ynF1tWr3q4qPQnJ63xF1dPPvIEjebkhc="
USAJOBS_API_URL = "https://data.usajobs.gov/api/Search"

# --------------- Adzuna API Configuration ---------------
ADZUNA_API_ID = "9d1bc7b6"
ADZUNA_API_KEY = "54a9e1822eb536ed93013900a73a68c3"
ADZUNA_API_URL = "https://api.adzuna.com/v1/api/jobs/us/search/1"

# ---------------- HARD-CODED SEARCH PARAMETERS ----------------
SEARCH_KEYWORDS = ["Software Engineer", "Data Scientist"]
SEARCH_SKILLS = ["Python", "SQL", "Machine Learning"]
SEARCH_LOCATION = "United States"
RESULTS_PER_PAGE = 25

In [3]:
def delete_tables(tables):
    conn = sqlite3.connect("job_recommendations.db")
    cursor = conn.cursor()

    try:
        for table in tables:
            cursor.execute(f"DROP TABLE IF EXISTS {table};")
        
        conn.commit()
        print("Tables deleted successfully!")
    except sqlite3.Error as e:
        print(f"Error deleting tables: {e}")
    
    conn.close()


In [3]:
# ["USER", "USER_SKILL", "USER_KEYWORD", "RECOMMEND", "KEYWORD", "SKILL", "JOB_SKILL", "JOB"]

In [4]:
# ---------------- VIEW FIRST N RECORDS FROM TABLE ----------------
def view_first_records(table_name, limit=5):
    """ Fetches and displays the first N records from a table """
    conn = sqlite3.connect("job_recommendations.db")
    cursor = conn.cursor()

    try:
        cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit}")  # Fetch first N records
        rows = cursor.fetchall()

        if rows:
            column_names = [description[0] for description in cursor.description]
            print(f"\n---- Showing First {limit} Records from {table_name} ----")
            for row in rows:
                print(dict(zip(column_names, row)))  # Display records as dictionaries
        else:
            print(f"\nNo records found in {table_name}.")

    except sqlite3.Error as e:
        print(f"Error viewing table {table_name}: {e}")

    conn.close()

# ---------------- VIEW LAST N RECORDS FROM TABLE ----------------
def view_last_records(table_name, limit=5):
    """ Fetches and displays the last N records from a table """
    conn = sqlite3.connect("job_recommendations.db")
    cursor = conn.cursor()

    try:
        cursor.execute(f"SELECT * FROM {table_name} ORDER BY ROWID DESC LIMIT {limit};")  # Fetch last N records
        rows = cursor.fetchall()

        if rows:
            column_names = [description[0] for description in cursor.description]
            print(f"\n---- Showing Last {limit} Records from {table_name} ----")
            for row in rows:
                print(dict(zip(column_names, row)))  # Display records as dictionaries
        else:
            print(f"\nNo records found in {table_name}.")

    except sqlite3.Error as e:
        print(f"Error viewing table {table_name}: {e}")

    conn.close()
    
def get_table_length(table_name):
    """ Returns the number of rows in a given table """
    conn = sqlite3.connect("job_recommendations.db")
    cursor = conn.cursor()

    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]

    conn.close()
    return count

In [ ]:
# ---------------- DATABASE CONNECTION ----------------
def create_connection():
    return sqlite3.connect("job_recommendations.db")

# ---------------- CREATE TABLES ----------------
def create_tables():
    conn = create_connection()
    cursor = conn.cursor()

    # USER TABLE (Now includes USER_USERNAME as UNIQUE)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS USER (
            USER_ID TEXT PRIMARY KEY, 
            USER_USERNAME TEXT UNIQUE NOT NULL,
            USER_RESUME TEXT
        );
    """)

    # JOB TABLE
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS JOB (
            JOB_ID TEXT PRIMARY KEY, 
            JOB_TITLE TEXT NOT NULL, 
            JOB_COMPANY TEXT NOT NULL, 
            JOB_LOCATION TEXT NOT NULL, 
            JOB_SALARY FLOAT(10,2), 
            JOB_DATEPOSTED TEXT NOT NULL, 
            JOB_DESCRIPTION TEXT, 
            JOB_CONTACTEMAIL TEXT, 
            JOB_REQUIREMENT TEXT, 
            JOB_URL TEXT NOT NULL, 
            JOB_API TEXT NOT NULL, 
            JOB_RECOMMENDED INTEGER DEFAULT 0, 
            JOB_REVIEWCOUNT INTEGER DEFAULT 0, 
            JOB_REVIEWRATING FLOAT DEFAULT 0.0
        );
    """)

    # KEYWORD TABLE
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS KEYWORD (
            KEYWORD_ID TEXT PRIMARY KEY, 
            KEYWORD_NAME TEXT UNIQUE NOT NULL
        );
    """)

    # SKILL TABLE
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS SKILL (
            SKILL_ID TEXT PRIMARY KEY, 
            SKILL_NAME TEXT UNIQUE NOT NULL
        );
    """)

    # USER_KEYWORD (Bridge Table)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS USER_KEYWORD (
            USER_ID TEXT, 
            KEYWORD_ID TEXT, 
            PRIMARY KEY (USER_ID, KEYWORD_ID)
        );
    """)

    # USER_SKILL (Bridge Table)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS USER_SKILL (
            USER_ID TEXT, 
            SKILL_ID TEXT, 
            PRIMARY KEY (USER_ID, SKILL_ID)
        );
    """)

    # JOB_SKILL (Bridge Table)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS JOB_SKILL (
            JOB_ID TEXT, 
            SKILL_ID TEXT, 
            PRIMARY KEY (JOB_ID, SKILL_ID)
        );
    """)

    # RECOMMEND TABLE (Replaces SEARCH table)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS RECOMMEND (
            USER_ID TEXT,
            JOB_ID TEXT,
            REC_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            REC_RATED INTEGER DEFAULT 0,  -- 0 = Not Rated, 1 = Rated
            REC_RATING INTEGER CHECK (REC_RATING BETWEEN 1 AND 5) DEFAULT NULL,  -- User rating (1-5 or NULL)
            PRIMARY KEY (USER_ID, JOB_ID, REC_DATE),
            FOREIGN KEY (USER_ID) REFERENCES USER (USER_ID),
            FOREIGN KEY (JOB_ID) REFERENCES JOB (JOB_ID)
        );
    """)

    conn.commit()
    conn.close()

# ---------------- GENERATE UNIQUE IDs ----------------
def generate_id(prefix):
    """ Generates a unique ID with a given prefix (e.g., U12345, S67890) """
    return f"{prefix}{random.randint(10000, 99999)}"

def generate_job_id(job_title, job_company, job_dateposted):
    """ Ensures unique JOB_ID based on job title, company, and date posted """
    formatted_title = job_title.replace(" ", "")
    formatted_company = job_company.replace(" ", "")

    try:
        formatted_date = datetime.strptime(job_dateposted[:10], "%Y-%m-%d").strftime("%d-%m-%y")
    except ValueError:
        formatted_date = "unknown"

    return f"{formatted_title}{formatted_company}{formatted_date}"

# ---------------- INSERT INTO TABLES ----------------
def insert_user(username, resume_text=None):
    """ Inserts a new user with a unique username and an optional resume """
    conn = create_connection()
    cursor = conn.cursor()

    # Check if username exists
    cursor.execute("SELECT USER_ID FROM USER WHERE USER_USERNAME = ?", (username,))
    existing_user = cursor.fetchone()

    if existing_user:
        print(f"Username '{username}' already exists. Using existing USER_ID: {existing_user[0]}")
        conn.close()
        return existing_user[0]  # Return existing user ID

    # Generate a new unique USER_ID
    user_id = generate_id("U")
    
    cursor.execute("INSERT INTO USER (USER_ID, USER_USERNAME, USER_RESUME) VALUES (?, ?, ?)", 
                   (user_id, username, resume_text))

    conn.commit()
    conn.close()
    return user_id

def insert_skills():
    skills = SEARCH_SKILLS  # Using predefined dropdown values
    conn = create_connection()
    cursor = conn.cursor()

    for skill in skills:
        skill_id = generate_id("S")
        cursor.execute("INSERT OR IGNORE INTO SKILL (SKILL_ID, SKILL_NAME) VALUES (?, ?)", (skill_id, skill))

    conn.commit()
    conn.close()

def assign_user_skills(user_id):
    conn = create_connection()
    cursor = conn.cursor()

    cursor.execute("SELECT SKILL_ID FROM SKILL WHERE SKILL_NAME IN ({})".format(",".join("?" * len(SEARCH_SKILLS))),
                   SEARCH_SKILLS)
    skill_ids = [row[0] for row in cursor.fetchall()]

    user_skills = [(user_id, skill_id) for skill_id in skill_ids]
    cursor.executemany("INSERT OR IGNORE INTO USER_SKILL (USER_ID, SKILL_ID) VALUES (?, ?)", user_skills)

    conn.commit()
    conn.close()

def insert_keyword(keyword):
    conn = create_connection()
    cursor = conn.cursor()

    cursor.execute("SELECT KEYWORD_ID FROM KEYWORD WHERE KEYWORD_NAME = ?", (keyword,))
    existing = cursor.fetchone()

    if existing:
        keyword_id = existing[0]
    else:
        keyword_id = generate_id("K")
        cursor.execute("INSERT INTO KEYWORD (KEYWORD_ID, KEYWORD_NAME) VALUES (?, ?)", (keyword_id, keyword))

    conn.commit()
    conn.close()
    return keyword_id

def insert_user_keyword(user_id, keyword_id):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT OR IGNORE INTO USER_KEYWORD (USER_ID, KEYWORD_ID) VALUES (?, ?)", (user_id, keyword_id))
    conn.commit()
    conn.close()
    
def update_job_recommendation_stats(job_id):
    """ Updates JOB_RECOMMENDED, JOB_REVIEWCOUNT, and JOB_REVIEWRATING for a given job """
    conn = create_connection()
    cursor = conn.cursor()

    # Get total recommendations for the job
    cursor.execute("SELECT COUNT(*) FROM RECOMMEND WHERE JOB_ID = ?", (job_id,))
    total_recommended = cursor.fetchone()[0] or 0  # Default to 0 if no rows found

    # Get total review count (where REC_RATED = 1)
    cursor.execute("SELECT SUM(REC_RATED) FROM RECOMMEND WHERE JOB_ID = ?", (job_id,))
    total_review_count = cursor.fetchone()[0] or 0  # Default to 0 if no reviews

    # Get average review rating, handle division by zero
    cursor.execute("SELECT AVG(REC_RATING) FROM RECOMMEND WHERE JOB_ID = ? AND REC_RATING IS NOT NULL", (job_id,))

    avg_review_rating = cursor.fetchone()[0]  # Can be None if no ratings

    avg_review_rating = round(avg_review_rating, 2) if avg_review_rating is not None else 0.0  # Default to 0.0

    # Update the JOB table
    cursor.execute("""
        UPDATE JOB 
        SET JOB_RECOMMENDED = ?, 
            JOB_REVIEWCOUNT = ?, 
            JOB_REVIEWRATING = ?
        WHERE JOB_ID = ?
    """, (total_recommended, total_review_count, avg_review_rating, job_id))

    conn.commit()
    conn.close()
    
def insert_recommendation(user_id, job_id):
    """ Inserts a job recommendation for a user and updates job stats """
    conn = create_connection()
    cursor = conn.cursor()

    try:
        cursor.execute("""
            INSERT INTO RECOMMEND (USER_ID, JOB_ID) 
            VALUES (?, ?)
        """, (user_id, job_id))
        
        conn.commit()
        update_job_recommendation_stats(job_id)  # Update JOB stats after insert
    except sqlite3.IntegrityError:
        print(f"⚠️ Job {job_id} was already recommended to user {user_id}.")

    conn.close()

def insert_job_skills(job_id):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT SKILL_ID FROM SKILL")
    skill_ids = [row[0] for row in cursor.fetchall()]
    job_skills = [(job_id, skill_id) for skill_id in skill_ids]
    cursor.executemany("INSERT OR IGNORE INTO JOB_SKILL (JOB_ID, SKILL_ID) VALUES (?, ?)", job_skills)
    conn.commit()
    conn.close()

def insert_job_data(job_list):
    conn = create_connection()
    cursor = conn.cursor()

    for job in job_list:
        cursor.execute("SELECT JOB_ID FROM JOB WHERE JOB_ID = ?", (job["id"],))
        if cursor.fetchone() is None:
            cursor.execute("""
                INSERT INTO JOB (JOB_ID, JOB_TITLE, JOB_COMPANY, JOB_LOCATION, JOB_SALARY, JOB_DATEPOSTED,
                                 JOB_DESCRIPTION, JOB_CONTACTEMAIL, JOB_REQUIREMENT, JOB_URL, JOB_API,
                                 JOB_RECOMMENDED, JOB_REVIEWCOUNT, JOB_REVIEWRATING)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (job["id"], job["title"], job["company"], job["location"], job["salary"],
                  job["date_posted"], job["description"], job["contact_email"],
                  job["requirements"], job["url"], job["api_source"], 0, 0, 0.0))

    conn.commit()
    conn.close()
    
def rate_recommendation(user_id, job_id, rating):
    """ Updates a recommended job with a user rating (1-5) and updates job stats """
    if rating < 1 or rating > 5:
        print("⚠️ Invalid rating. Please enter a value between 1 and 5.")
        return

    conn = create_connection()
    cursor = conn.cursor()

    # Ensure the job was actually recommended before updating
    cursor.execute("SELECT * FROM RECOMMEND WHERE USER_ID = ? AND JOB_ID = ?", (user_id, job_id))
    if cursor.fetchone():
        cursor.execute("""
            UPDATE RECOMMEND 
            SET REC_RATED = 1, REC_RATING = ? 
            WHERE USER_ID = ? AND JOB_ID = ?
        """, (rating, user_id, job_id))

        conn.commit()
        update_job_recommendation_stats(job_id)  # Update JOB stats after rating
        print(f"✅ Rating {rating} saved for job {job_id}!")
    else:
        print(f"⚠️ Job {job_id} was not recommended to user {user_id}.")

    conn.close()

def get_user_resume(user_id):
    """ Fetch user resume text from database """
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT USER_RESUME FROM USER WHERE USER_ID = ?", (user_id,))
    resume = cursor.fetchone()
    conn.close()
    return resume[0] if resume else None
    
# ---------------- FETCH JOB LISTINGS ----------------
def fetch_usajobs(keyword):
    """ Fetch jobs from USAJobs API with error handling """
    headers = {
        "Host": "data.usajobs.gov",
        "User-Agent": "YourEmail@example.com",
        "Authorization-Key": USAJOBS_API_KEY
    }
    params = {"Keyword": keyword, "LocationName": SEARCH_LOCATION, "ResultsPerPage": RESULTS_PER_PAGE}

    response = requests.get(USAJOBS_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        try:
            jobs = response.json().get("SearchResult", {}).get("SearchResultItems", [])
            job_list = []
            for job in jobs:
                job_data = job.get("MatchedObjectDescriptor", {})

                # Ensure `description` is a string (USAJobs sometimes returns a list)
                description = job_data.get("UserArea", {}).get("Details", {}).get("MajorDuties", "N/A")
                if isinstance(description, list):
                    description = "\n".join(description)  # Convert list to string
                remuneration = job_data.get("PositionRemuneration", [{}])
                
                job_list.append({
                    "id": generate_job_id(job_data.get("PositionTitle", ""), job_data.get("OrganizationName", ""), job_data.get("PublicationStartDate", "")),
                    "title": job_data.get("PositionTitle", "N/A"),
                    "company": job_data.get("OrganizationName", "N/A"),
                    "location": job_data.get("PositionLocationDisplay", "N/A"),
                    "salary": remuneration[0].get("MinimumRange", "N/A") if remuneration else "N/A",
                    "date_posted": job_data.get("PublicationStartDate", "N/A"),
                    "description": description,
                    "contact_email": job_data.get("UserArea", {}).get("Details", {}).get("AgencyContactEmail", "N/A"),
                    "requirements": job_data.get("QualificationSummary", "N/A"),
                    "url": job_data.get("PositionURI", "N/A"),
                    "api_source": "USAJobs"
                })
            return job_list
        except Exception as e:
            print("Error parsing USAJobs response:", e)
            return []
    else:
        print(f"USAJobs API Error {response.status_code}: {response.text}")
        return []

def fetch_adzuna_jobs(keyword):
    """ Fetch jobs from Adzuna API with better error handling """
    params = {
        "app_id": ADZUNA_API_ID, "app_key": ADZUNA_API_KEY,
        "results_per_page": RESULTS_PER_PAGE, "what": keyword, "where": SEARCH_LOCATION
    }

    response = requests.get(ADZUNA_API_URL, params=params)
    
    if response.status_code == 200:
        try:
            jobs = response.json().get("results", [])
            job_list = []
            for job in jobs:
                job_list.append({
                    "id": generate_job_id(job.get("title", ""), job.get("company", {}).get("display_name", "N/A"), job.get("created", "")),
                    "title": job.get("title", "N/A"),
                    "company": job.get("company", {}).get("display_name", "N/A"),
                    "location": job.get("location", {}).get("display_name", "N/A"),
                    "salary": job.get("salary_min", "N/A"),
                    "date_posted": job.get("created", "N/A"),
                    "description": job.get("description", "N/A"),
                    "contact_email": "N/A",
                    "requirements": "N/A",
                    "url": job.get("redirect_url", "N/A"),
                    "api_source": "Adzuna"
                })
            return job_list
        except Exception as e:
            print("Error parsing Adzuna response:", e)
            return []
    else:
        print(f"Adzuna API Error {response.status_code}: {response.text}")
        return []

# ---------------- COMBINE JOB LISTINGS ----------------
def get_combined_jobs(keyword):
    """ Fetches jobs from both APIs and combines results """
    all_jobs = fetch_usajobs(keyword) + fetch_adzuna_jobs(keyword)
    return all_jobs

# ---------------- READ RESUME FROM .DOCX ----------------
def read_resume(file_path):
    """ Reads text from a .docx resume file """
    try:
        doc = Document(file_path)
        return "\n".join([para.text.strip() for para in doc.paragraphs if para.text.strip()])
    except Exception as e:
        print(f"Error reading resume file: {e}")
        return None

# ---------------- STORE RESUME IN DATABASE ----------------
def store_resume(user_id, resume_text):
    """ Stores resume text into USER table """
    if not resume_text:
        print("No resume text extracted.")
        return
    
    conn = create_connection()
    cursor = conn.cursor()
    
    cursor.execute("UPDATE USER SET USER_RESUME = ? WHERE USER_ID = ?", (resume_text, user_id))

    conn.commit()
    conn.close()
    print(f"Resume stored successfully for USER_ID: {user_id}")

# ---------------- AI MODEL: RECOMMEND JOBS ----------------
def get_jobs():
    """ Fetch job listings from the database """
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT JOB_ID, JOB_TITLE, JOB_DESCRIPTION FROM JOB")
    jobs = cursor.fetchall()
    conn.close()
    return jobs

def get_seen_jobs(user_id):
    """ Fetches JOB_IDs that have already been recommended to the user """
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT JOB_ID FROM RECOMMEND WHERE USER_ID = ?", (user_id,))
    seen_jobs = {row[0] for row in cursor.fetchall()}  # Store JOB_IDs in a set for fast lookup
    conn.close()
    return seen_jobs

def recommend_jobs(user_id, top_n=5):
    """ Uses AI to recommend jobs based on resume similarity, ensuring only new top N jobs are recommended """

    # Step 1: Fetch user resume text
    resume_text = get_user_resume(user_id)
    if not resume_text:
        print("⚠️ No resume found for this user. Cannot generate recommendations.")
        return []

    # Step 2: Fetch job listings
    job_data = get_jobs()
    if not job_data:
        print("⚠️ No jobs available for recommendation.")
        return []

    job_ids, job_titles, job_descriptions = zip(*job_data)

    # Step 3: Ensure job descriptions are valid strings
    job_descriptions = [desc if isinstance(desc, str) else "" for desc in job_descriptions]

    # Step 4: Convert text data into vectors using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([resume_text] + list(job_descriptions))

    # Step 5: Compute similarity between resume and jobs
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

    # Step 6: Rank jobs by similarity score
    ranked_jobs = sorted(zip(job_ids, job_titles, similarity_scores), key=lambda x: x[2], reverse=True)

    # Step 7: Fetch jobs already recommended to this user
    seen_jobs = get_seen_jobs(user_id)

    # Step 8: Filter out already recommended jobs
    new_recommendations = [(job_id, title, score) for job_id, title, score in ranked_jobs if job_id not in seen_jobs]

    # Step 9: Get only the next top N jobs
    recommended_jobs = new_recommendations[:top_n]

    # Step 10: Display and store recommendations
    if recommended_jobs:
        print("\n🔥 Top Recommended Jobs 🔥")
        for job_id, title, score in recommended_jobs:
            print(f"📌 {title} (Score: {score:.4f})")
            insert_recommendation(user_id, job_id)  # Insert only new recommendations
    else:
        print("⚠️ No new job recommendations available.")

    return recommended_jobs

def regenerate_recommendations(user_id, top_n=5):
    """ Allows the user to get new recommendations different from previous ones """
    print("\n🔄 Regenerating Recommendations...")
    
    # Remove the last recommended jobs
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("DELETE FROM RECOMMEND WHERE USER_ID = ?", (user_id,))
    conn.commit()
    conn.close()

    # Get new job recommendations
    return recommend_jobs(user_id, top_n)

# ---------------- RUN TEST ----------------
if __name__ == "__main__":
    # Step 1: Create tables
    create_tables()
    insert_skills() #hard coded

    # Step 2: Hardcoded user details
    hardcoded_username = "test_user"
    user_id = insert_user(hardcoded_username)

    # Step 3: Read and store resume from folder
    resume_path = r"Uploads\Elijah Mitchell - Resume.docx"  # Hard coded
    if os.path.exists(resume_path):
        resume_text = read_resume(resume_path)
        store_resume(user_id, resume_text)
    else:
        print(f"⚠️ Resume file not found: {resume_path}")

    # Step 4: Assign hardcoded skills
    hardcoded_skills = ["Python", "Machine Learning", "SQL"]
    assign_user_skills(user_id)

    # Step 5: Assign hardcoded keywords
    hardcoded_keywords = ["Software Engineer", "Data Scientist"]
    for keyword in hardcoded_keywords:
        keyword_id = insert_keyword(keyword)
        insert_user_keyword(user_id, keyword_id)

    # Step 6: Fetch job listings from APIs
    all_jobs = set()
    for keyword in hardcoded_keywords:
        jobs = get_combined_jobs(keyword)  # Fetch fresh data from APIs

        for job in jobs:
            if job["id"] not in all_jobs:
                all_jobs.add(job["id"])
                insert_job_data([job])  # Insert job into database
                insert_job_skills(job["id"])  # Assign skills to job

    # Step 7: AI-Powered Job Recommendations (Fix: Insert only top 5)
    print("\n🎯 AI Recommended Jobs for You:\n")
    recommend_jobs(user_id, top_n=5)  # FIX: Now only inserts 5 jobs

    print(f"\n✅ Test completed. {len(all_jobs)} unique jobs inserted & recommended.")


Username 'test_user' already exists. Using existing USER_ID: U90925
Resume stored successfully for USER_ID: U90925

🎯 AI Recommended Jobs for You:


🔥 Top Recommended Jobs 🔥
📌 ENGINEER/SCIENTIST (Score: 0.1577)
📌 DATA SCIENTIST (Score: 0.1541)
📌 Data Scientist - Workforce Analytics (Score: 0.1494)
📌 Senior Data Scientist (Score: 0.1476)
📌 Senior Software Engineer/LabVIEW (Score: 0.1451)

✅ Test completed. 95 unique jobs inserted & recommended.


In [32]:
view_first_records("USER_KEYWORD", 10)


---- Showing First 10 Records from USER_KEYWORD ----
{'USER_ID': 'U45965', 'KEYWORD_ID': 'K71968'}
{'USER_ID': 'U45965', 'KEYWORD_ID': 'K78526'}
{'USER_ID': 'U45965', 'KEYWORD_ID': 'K77255'}


In [23]:
for t in ["USER", "USER_SKILL", "USER_KEYWORD", "RECOMMEND", "KEYWORD", "SKILL", "JOB_SKILL", "JOB"]:
    print(get_table_length(t))

1
5
8
5
2
5
470
94


In [14]:
delete_tables(["USER", "USER_SKILL", "USER_KEYWORD", "RECOMMEND", "KEYWORD", "SKILL", "JOB_SKILL", "JOB"])

Tables deleted successfully!


In [7]:
view_first_records("JOB")


No records found in JOB.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ✅ Load BLS Occupation Data
bls_data = pd.read_csv("bls_occupation_profiles.csv")  # Ensure this file is in your project

def recommend_career_goal(user_id):
    """Analyzes user's resume and skills to suggest a career goal from BLS data."""
    
    # Step 1: Fetch user data
    resume_text = get_user_resume(user_id)
    if not resume_text:
        return {"error": "No resume found for this user."}
    
    user_skills = get_user_skills(user_id)  # Fetches user skills from the DB
    user_keywords = get_user_keywords(user_id)  # Fetches user keywords from DB

    # Step 2: Process BLS Data
    bls_data["combined_info"] = (
        bls_data["title"] + " " + 
        bls_data["description"] + " " +
        bls_data["education"] + " " + 
        bls_data["experience"] + " " +
        bls_data["salary"] + " " +
        bls_data["job_outlook"]
    )

    # Step 3: Compute Similarity
    vectorizer = TfidfVectorizer(stop_words="english")
    corpus = [resume_text + " " + " ".join(user_skills) + " ".join(user_keywords)] + list(bls_data["combined_info"])
    tfidf_matrix = vectorizer.fit_transform(corpus)
    
    # Step 4: Rank Career Matches
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    bls_data["similarity"] = similarity_scores
    best_match = bls_data.sort_values(by="similarity", ascending=False).iloc[0]

    # Step 5: Generate Career Roadmap
    career_goal = {
        "title": best_match["title"],
        "description": best_match["description"],
        "average_salary": best_match["salary"],
        "education_needed": best_match["education"],
        "experience_needed": best_match["experience"],
        "job_outlook": best_match["job_outlook"],
        "roadmap": f"To become a {best_match['title']}, you need {best_match['education']} "
                   f"and {best_match['experience']} experience. Consider getting certifications in relevant fields. "
                   f"The job outlook is {best_match['job_outlook']}, with an average salary of {best_match['salary']}."
    }

    return career_goal
